# K - nearest neighbors algorithm

$ \text{- Step 1: Calculate Euclidean Distance}$

Caculate the distance between two rows in a dataset.

$$ distant = \sqrt{\sum_{n=1} ^{i} (x_{1_{i}} - x_{2_{i}}) ^ 2}$$

$ \text{- Step 2: Get nearest neighbors}$

Neighbors for a new piece of data in the dataset are the k closest instances, as defined by our distance measure.

$ \text{- Step 3: Make predictions}$

The most similar neighbors collected from the training dataset can be used to make predictions.

In the case of classification, we can return the most represented class among the neighbors.





In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import neighbors, datasets

In [2]:
iris = datasets.load_iris()
iris_X = iris.data
iris_y = iris.target
print('Number of classes: %d' %len(np.unique(iris_y)))
print('Number of data points: %d' %len(iris_y))


# X0 = iris_X[iris_y == 0,:]
# print('\nSamples from class 0:\n', X0[:5,:])

# X1 = iris_X[iris_y == 1,:]
# print('\nSamples from class 1:\n', X1[:5,:])

# X2 = iris_X[iris_y == 2,:]
# print('\nSamples from class 2:\n', X2[:5,:])


Number of classes: 3
Number of data points: 150


In [3]:
df = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                     columns= iris['feature_names'] + ['target'])
df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2.0
146,6.3,2.5,5.0,1.9,2.0
147,6.5,3.0,5.2,2.0,2.0
148,6.2,3.4,5.4,2.3,2.0


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   sepal length (cm)  150 non-null    float64
 1   sepal width (cm)   150 non-null    float64
 2   petal length (cm)  150 non-null    float64
 3   petal width (cm)   150 non-null    float64
 4   target             150 non-null    float64
dtypes: float64(5)
memory usage: 6.0 KB


In [67]:
df.iloc[0]

sepal length (cm)    5.1
sepal width (cm)     3.5
petal length (cm)    1.4
petal width (cm)     0.2
target               0.0
Name: 0, dtype: float64

In [65]:
df.iloc[::,:-1:]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [12]:
def euclidean_distance(row1, row2):
    """caculate the distance between two points

    Args:
        row1 (list): coordinate of point
        row2 (list): coordinate of point

    Returns:
        distance (float): distance between two points
    """

    distance = ((row1 - row2) ** 2).sum(axis=0) ** 0.5
    return distance

In [13]:

row0 = df.iloc[0]
for i in range(len(df)):
    distance = euclidean_distance(row0[:-1:], df.iloc[i][:-1:])
    print(distance)


0.0
0.5385164807134502
0.509901951359278
0.648074069840786
0.1414213562373093
0.6164414002968979
0.5196152422706632
0.17320508075688762
0.9219544457292882
0.4690415759823426
0.37416573867739483
0.3741657386773941
0.5916079783099616
0.9949874371066197
0.8831760866327848
1.1045361017187267
0.5477225575051664
0.09999999999999998
0.7416198487095667
0.33166247903553986
0.4358898943540679
0.30000000000000016
0.648074069840786
0.46904157598234303
0.5916079783099616
0.5477225575051662
0.316227766016838
0.14142135623730995
0.14142135623730995
0.53851648071345
0.5385164807134504
0.3872983346207423
0.6244997998398396
0.8062257748298554
0.4582575694955836
0.37416573867739383
0.41231056256176635
0.24494897427831727
0.866025403784438
0.14142135623730964
0.17320508075688743
1.3490737563232043
0.7681145747868601
0.45825756949558394
0.6164414002968975
0.5916079783099616
0.3605551275463989
0.58309518948453
0.30000000000000027
0.22360679774997896
4.003748243833521
3.6166282640050254
4.164132562731403
3.0

In [117]:
def get_neighbors(train_set, test_row, num_neighbors):
    """Get K nearest points to our point

    Args:
        train_set (list (m,n)): data set
        test_row (list (1,n)): out point
        num_neighbors (int): how many neighbor points we want to get

    Returns:
        neighbors (list (k,n)): K nearest neighbors
    """

    distances = []

    # caculate the distance between each pair of point and add them to the list
    for i in range(train_set.shape[0]):
        train_row = train_set.iloc[i]
        dist = euclidean_distance(test_row[:-1:], train_row[:-1:])
        if dist != 0:
            distances.append((train_row, dist))
    
    # soft bases on the distance 
    distances.sort(key=lambda tup: tup[1])
    neighbors = []

    # get K nearest points
    for i in range(num_neighbors):
        neighbors.append(distances[i][0])
    return neighbors

In [115]:

neighbors = get_neighbors(df, df.iloc[0], 3)
for neighbor in neighbors:
    print(neighbor)

sepal length (cm)    5.0
sepal width (cm)     3.5
petal length (cm)    1.3
petal width (cm)     0.3
target               0.0
Name: 40, dtype: float64
sepal length (cm)    5.0
sepal width (cm)     3.6
petal length (cm)    1.4
petal width (cm)     0.2
target               0.0
Name: 4, dtype: float64
sepal length (cm)    5.1
sepal width (cm)     3.4
petal length (cm)    1.5
petal width (cm)     0.2
target               0.0
Name: 39, dtype: float64


In [118]:
def predict_classification(train, test_set, num_neighbors):
    """Give the predict bases on the most present class/specie

    Args:
        train (_type_): _description_
        test_row (_type_): _description_
        num_neighbors (_type_): _description_

    Returns:
        _type_: _description_
    """

    prediction_list = []
    for i in range(test_set.shape[0]):
        neighbors = get_neighbors(train, test_set.iloc[i], num_neighbors)
        output_values = [row[-1] for row in neighbors]
        prediction = max(set(output_values), key=output_values.count)
        prediction_list.append(prediction)
    return prediction_list

In [125]:

prediction = predict_classification(df, df.iloc[:3:], 3)
print('Expected {}, Got {}.'.format(df.iloc[::,-1][:3:], prediction))

Expected 0    0.0
1    0.0
2    0.0
Name: target, dtype: float64, Got [0.0, 0.0, 0.0].


In [66]:
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

#### testing 

In [127]:
df = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                     columns= iris['feature_names'] + ['target'])

dataset = df.copy(deep=True)
dataset = dataset.sample(frac=1)
dataset

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
129,7.2,3.0,5.8,1.6,2.0
30,4.8,3.1,1.6,0.2,0.0
29,4.7,3.2,1.6,0.2,0.0
128,6.4,2.8,5.6,2.1,2.0
49,5.0,3.3,1.4,0.2,0.0
...,...,...,...,...,...
77,6.7,3.0,5.0,1.7,1.0
46,5.1,3.8,1.6,0.2,0.0
144,6.7,3.3,5.7,2.5,2.0
44,5.1,3.8,1.9,0.4,0.0


In [133]:
train_set = dataset.iloc[:100:].reset_index()
test_set = dataset.iloc[100::].reset_index()

In [137]:
pred_list = predict_classification(train_set, test_set, 5)
print('Expected {}\n Got {}.'.format(test_set.iloc[::,-1].to_list(), pred_list))

Expected [1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 2.0, 0.0, 1.0, 2.0, 0.0, 0.0, 1.0, 1.0, 1.0, 2.0, 0.0, 0.0, 2.0, 0.0, 1.0, 2.0, 0.0, 1.0, 2.0, 2.0, 2.0, 2.0, 1.0, 0.0, 1.0, 2.0, 0.0, 2.0, 2.0, 1.0, 2.0, 0.0, 1.0, 1.0, 2.0, 0.0, 2.0, 0.0, 2.0, 1.0, 0.0, 2.0, 0.0, 2.0]
 Got [1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 2.0, 0.0, 1.0, 2.0, 0.0, 0.0, 1.0, 1.0, 1.0, 2.0, 0.0, 0.0, 2.0, 0.0, 1.0, 2.0, 0.0, 1.0, 2.0, 2.0, 2.0, 2.0, 1.0, 0.0, 1.0, 2.0, 0.0, 2.0, 2.0, 1.0, 2.0, 0.0, 1.0, 1.0, 2.0, 0.0, 2.0, 0.0, 2.0, 1.0, 0.0, 2.0, 0.0, 2.0].


In [138]:
print(accuracy_metric(test_set.iloc[::,-1].to_list(), pred_list))

100.0
